## Importing utilities and defining global variables

In [1]:
from utilities_IGNOBEL import *
with open('credential.json','r') as f:
    cred = json.load(f)
cluster = MongoClient(cred['cred'])
db = cluster['Game']
collection = db['Players']
import glob 
import re, time

from datetime import datetime

F,S,G,C,V,M,CI = 0,0,0,0,0,0,0

In [2]:
season = '2021-22'
giornata = 38


## Automated update of stats using background routine

running the cell below will start a background infinite loop that updates database with new stats every 15 min. The global variable `missing_ids` contains the list of ids of the new players that need to be added to the db

In [3]:
import parsl
from parsl.app.app import python_app
from parsl.configs.local_threads import config

try:
    parsl.load(config)
except:
    pass
last_update_time = None
missing_ids=[]
stats = []
last_personal_update = []

errors=[]

@python_app
def automat_bulk_update_stats(season=season, primavera=True, db=db):
    global missing_ids, last_update_time, stats, last_personal_update
    while True:
        try:
            missing_ids, stats = bulk_update_stats(season, primavera, db)
            if len(missing_ids):
                missing_pl = info_missing_players(missing_ids, stats)
                add_multi_pl(missing_ids, stats, missing_pl)

            last_personal_update = bulk_update_personal_info(collection,stats)             
            last_update_time = datetime.now().strftime('%Y-%m-%d;%H:%M')
        except Exception as e:
            errors.append('recorded error %s at time %s'%(e,datetime.now().strftime('%Y-%m-%d;%H:%M')))
        time.sleep(600)
    
out=automat_bulk_update_stats()

In [17]:
last_update_time

'2022-05-26;09:49'

In [18]:
last_update_time

'2022-05-26;09:49'

In [14]:
errors

[]

The cell below is used to force update the database

In [10]:
#bulk_update_stats(season=season, primavera=True, db=db, verbose=True)

## IGNOBEL db update

In [5]:
"""Usare queste rose se FG non ha quelle corrette, e.g. durante il mercato"""
#RoseSS = rose_complete()

#RoseFG = rose()

#labels = RoseFG.columns
#Rose = RoseSS[labels]

'Usare queste rose se FG non ha quelle corrette, e.g. durante il mercato'

In [21]:
R=rose()
tot=IGNOBEL_tot(giornata, R)
tot

,IGNORANZA EVERYWHERE,PDG 1908,I DISEREDATI,XYZ,MAINZ NA GIOIA,PALLA PAZZA,SOROS FC,AS 800A
Fantapunti Fatti,74.5,73.5,65.25,64.00,66.25,58.75,68.25,76.75
Fantapunti Subiti,73.5,74.5,64.00,65.25,58.75,66.25,76.75,68.25
Goal subiti,0.0,0.0,3.00,0.00,0.00,0.00,0.00,0.00
C. gialli,2.0,0.0,1.00,2.00,1.00,1.00,1.00,1.00
C. rossi,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00
Bonus Panchinari,2.0,0.5,4.50,0.75,1.50,4.50,0.00,0.00
Modificatore,1.0,1.5,1.00,0.00,0.00,0.00,0.00,0.00
tot Infortunati,1.0,5.0,2.00,3.00,1.00,3.00,2.00,4.00


In [22]:
create_results_post('2021-22', giornata, tot, post_to_mongo=True)
tot

Results added to MongoDB


,IGNORANZA EVERYWHERE,PDG 1908,I DISEREDATI,XYZ,MAINZ NA GIOIA,PALLA PAZZA,SOROS FC,AS 800A
Fantapunti Fatti,74.5,73.5,65.25,64.00,66.25,58.75,68.25,76.75
Fantapunti Subiti,73.5,74.5,64.00,65.25,58.75,66.25,76.75,68.25
Goal subiti,0.0,0.0,3.00,0.00,0.00,0.00,0.00,0.00
C_gialli,2.0,0.0,1.00,2.00,1.00,1.00,1.00,1.00
C_rossi,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00
Bonus Panchinari,2.0,0.5,4.50,0.75,1.50,4.50,0.00,0.00
Modificatore,1.0,1.5,1.00,0.00,0.00,0.00,0.00,0.00
tot Infortunati,1.0,5.0,2.00,3.00,1.00,3.00,2.00,4.00


This cell updates directly the database with the new stats, including quotations

## Creating local databases used by other notebook to add players to db (obsolete)